In [1]:
import os
os.chdir("C:/Users/zetru/OneDrive/Mémoire python/")
import numpy as np
import pandas as pd
import statsmodels.api as sm
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['figure.figsize'] = 16, 9
plt.rcParams['axes.grid'] = True
import seaborn as sns
sns.set_theme(style="whitegrid")

In [2]:
from statsmodels.iolib.summary2 import summary_col
from statsmodels.tsa.stattools import adfuller

In [3]:
df=pd.read_hdf('df_v5.h5','df_v5')
display(df)

,cp,id,nom_carburant,id_carburant,maj,valeur,litre_brut,type,latitude,longitude,ville,stations_rayon_5km,competition_geo
0,01,1000001,Gazole,1,2022-01-03,0.702500,0.435626,R,46.201,5.198,SAINT-DENIS-LèS-BOURG,5,3
1,01,1000001,Gazole,1,2022-01-05,0.692500,0.447949,R,46.201,5.198,SAINT-DENIS-LèS-BOURG,5,3
2,01,1000001,Gazole,1,2022-01-10,0.728333,0.452874,R,46.201,5.198,SAINT-DENIS-LèS-BOURG,5,3
3,01,1000001,Gazole,1,2022-01-17,0.739167,0.484043,R,46.201,5.198,SAINT-DENIS-LèS-BOURG,5,3
4,01,1000001,Gazole,1,2022-01-21,0.769167,0.497616,R,46.201,5.198,SAINT-DENIS-LèS-BOURG,5,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4537444,95,95870010,SP98,6,2022-12-23,0.856233,0.488413,R,48.936,2.206,Bezons,15,4
4537445,95,95870010,SP98,6,2022-12-27,0.874567,0.487324,R,48.936,2.206,Bezons,15,4
4537446,95,95870010,SP98,6,2022-12-28,0.874567,0.484175,R,48.936,2.206,Bezons,15,4
4537447,95,95870010,SP98,6,2022-12-29,0.879567,0.477600,R,48.936,2.206,Bezons,15,4


In [4]:
df['ln_PP']=np.log(df['valeur'])
df['ln_Brent']=np.log(df['litre_brut'])

In [5]:
df_Gazole_A=df[(df['nom_carburant']=='Gazole')&(df['type']=='A')]
df_Gazole_R=df[(df['nom_carburant']=='Gazole')&(df['type']=='R')]
df_SP95_A=df[(df['nom_carburant']=='SP95')&(df['type']=='A')]
df_SP95_R=df[(df['nom_carburant']=='SP95')&(df['type']=='R')]
df_E10_A=df[(df['nom_carburant']=='E10')&(df['type']=='A')]
df_E10_R=df[(df['nom_carburant']=='E10')&(df['type']=='R')]
df_SP98_A=df[(df['nom_carburant']=='SP98')&(df['type']=='A')]
df_SP98_R=df[(df['nom_carburant']=='SP98')&(df['type']=='R')]

### Relation de long-terme (Equation 1)

In [6]:
# Liste des DataFrames pour autoroutes et routes
dfs = [df_Gazole_A, df_Gazole_R, df_SP95_A, df_SP95_R, df_E10_A, df_E10_R, df_SP98_A, df_SP98_R]
df_autoroutes = [df_Gazole_A, df_SP95_A, df_E10_A, df_SP98_A]
df_routes = [df_Gazole_R, df_SP95_R, df_E10_R, df_SP98_R]
labels = ['Gazole', 'SP95', 'E10', 'SP98']

# Fonction pour effectuer la régression linéaire
def run_regression_LT(df):
    y = df['ln_PP']  # prix à la pompe HT
    X = df['ln_Brent']  # prix du Brent
    X = sm.add_constant(X)  # ajoute une constante au modèle
    model = sm.OLS(y, X)
    results = model.fit()
    df['residus'] = results.resid
    return results

# Exécuter la régression pour chaque DataFrame
results_autoroutes = [run_regression_LT(df) for df in df_autoroutes]
results_routes = [run_regression_LT(df) for df in df_routes]

# Créer le tableau récapitulatif des résultats
summary_table_autoroutes = summary_col(results_autoroutes, stars=True, model_names=labels,
                                       info_dict={'No. observations': lambda x: f"{int(x.nobs)}"})
summary_table_routes = summary_col(results_routes, stars=True, model_names=labels,
                                   info_dict={'No. observations': lambda x: f"{int(x.nobs)}"})

C:\Users\zetru\AppData\Local\Temp\ipykernel_32224\3382741123.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['residus'] = results.resid
C:\Users\zetru\AppData\Local\Temp\ipykernel_32224\3382741123.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['residus'] = results.resid
C:\Users\zetru\AppData\Local\Temp\ipykernel_32224\3382741123.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the

In [7]:
# Afficher les tableaux récapitulatifs
print("Résultats pour les routes:")
display(summary_table_routes)
print("\nRésultats pour les autoroutes:")
display(summary_table_autoroutes)

Résultats pour les routes:


,Gazole,SP95,E10,SP98
const,0.4907***,0.4149***,0.4056***,0.4515***
,(0.0004),(0.0006),(0.0003),(0.0003)
ln_Brent,0.8655***,0.9285***,0.9624***,0.9200***
,(0.0007),(0.0011),(0.0006),(0.0006)
R-squared,0.5379,0.6906,0.6976,0.6814
R-squared Adj.,0.5379,0.6906,0.6976,0.6814
No. observations,1286334,326889,1026934,1021550



Résultats pour les autoroutes:


,Gazole,SP95,E10,SP98
const,0.5283***,0.3937***,0.4666***,0.5070***
,(0.0013),(0.0105),(0.0013),(0.0013)
ln_Brent,0.7585***,0.8553***,0.8646***,0.8168***
,(0.0025),(0.0200),(0.0025),(0.0024)
R-squared,0.4860,0.6251,0.5648,0.5522
R-squared Adj.,0.4859,0.6247,0.5648,0.5522
No. observations,98226,1102,95499,93426


In [8]:
# Convertir le tableau récapitulatif en format LaTeX et le sauvegarder dans un fichier
def export_to_latex(summary, filename):
    try:
        # Utilisation de as_latex() pour obtenir la chaîne LaTeX
        latex_content = summary.as_latex()
        # Écriture du contenu dans un fichier .tex
        with open(filename, 'w') as file:
            file.write(latex_content)
        print(f"Le fichier {filename} a été créé avec succès.")
    except Exception as e:
        print(f"Une erreur est survenue lors de l'écriture du fichier LaTeX : {e}")

In [9]:
export_to_latex(summary_table_autoroutes, 'tex/LT_A.tex')
export_to_latex(summary_table_routes, 'tex/LT_R.tex')

Le fichier tex/LT_A.tex a été créé avec succès.
Le fichier tex/LT_R.tex a été créé avec succès.


### Test ADF sur les résidus de la relation de long-terme

In [10]:
results_adf = [adfuller(df['residus'],maxlag=0) for df in dfs]

results_adf

In [11]:
labels_adf = ['Gazole', 'Gazole', 'SP95', 'SP95', 'E10', 'E10', 'SP98', 'SP98']
typelabels = ['Autoroute', 'Route', 'Autoroute', 'Route', 'Autoroute', 'Route', 'Autoroute', 'Route']

# Construction du DataFrame des statistiques ADF
adf_stats = {
    'Carburant': labels_adf,
    'Type': typelabels,
    'ADF Statistic': [result[0] for result in results_adf],
    'p-value': [result[1] for result in results_adf],
    'Used Lag': [result[2] for result in results_adf],
    'Obs.': [result[3] for result in results_adf]
}

# Création du DataFrame à partir du dictionnaire de statistiques ADF
adf_results_df = pd.DataFrame(adf_stats)

# Affichage du DataFrame
adf_results_df

,Carburant,Type,ADF Statistic,p-value,Used Lag,Obs.
0,Gazole,Autoroute,-57.587979,0.000000e+00,0,98225
1,Gazole,Route,-264.070372,0.000000e+00,0,1286333
2,SP95,Autoroute,-11.269146,1.552432e-20,0,1101
3,SP95,Route,-203.621261,0.000000e+00,0,326888
4,E10,Autoroute,-56.464009,0.000000e+00,0,95498
5,E10,Route,-273.946010,0.000000e+00,0,1026933
6,SP98,Autoroute,-54.616996,0.000000e+00,0,93425
7,SP98,Route,-277.889663,0.000000e+00,0,1021549


In [12]:
adf_results_df.to_latex('tex/adf_results.tex',index=False)

### Relation de court-terme : Modèle à correction d'erreur sans asymétrie (Equation 2)

In [13]:
labels = ['Gazole', 'SP95', 'E10', 'SP98']
# Préparation des données pour la régression de correction d'erreur à court terme
def prepare_data_for_ecm(df):
    # Créer une copie profonde du DataFrame pour éviter les modifications sur le DataFrame original
    df = df.copy()
    
    # Obtenir l'index après le groupby pour pouvoir utiliser .loc plus tard
    df['id_index'] = df.groupby('id').ngroup()
    
    # Utiliser .loc pour effectuer les affectations
    df.loc[:, 'delta_ln_PP'] = df.groupby('id')['ln_PP'].transform(lambda x: x.diff())
    df.loc[:, 'delta_ln_Brent'] = df.groupby('id')['ln_Brent'].transform(lambda x: x.diff())
    
    df.loc[:, 'error_correction'] = df.groupby('id')['residus'].transform(lambda x: x.shift(1))
    
    df.loc[:, 'delta_ln_PP_1'] = df.groupby('id')['delta_ln_PP'].transform(lambda x: x.shift(1))
    df.loc[:, 'delta_ln_Brent_1'] = df.groupby('id')['delta_ln_Brent'].transform(lambda x: x.shift(1))
    df.loc[:, 'delta_ln_Brent_2'] = df.groupby('id')['delta_ln_Brent_1'].transform(lambda x: x.shift(1))
    
    # Suppression des NA générés par diff() et shift()
    df.dropna(subset=['delta_ln_PP', 'delta_ln_Brent', 'error_correction',
                      'delta_ln_PP_1', 'delta_ln_Brent_1', 'delta_ln_Brent_2'], inplace=True)
    
    # Supprimer la colonne temporaire 'id_index'
    df.drop(columns='id_index', inplace=True)
    
    return df

# Fonction pour effectuer la régression de correction d'erreur à court terme
def run_ecm(df):
    # Préparation des données
    df = prepare_data_for_ecm(df)
    
    # Définition des variables pour le modèle
    X_ecm = df[['delta_ln_Brent', 'delta_ln_Brent_1', 'delta_ln_Brent_2', 'delta_ln_PP_1', 'error_correction']]
    y_ecm = df['delta_ln_PP']
    
    # Ajustement du modèle ECM sans constante
    model_ecm = sm.OLS(y_ecm, X_ecm)
    results_ecm = model_ecm.fit()
    return results_ecm

# Application de la fonction de régression ECM à chaque DataFrame de sous-groupe
results_autoroutes_ecm = [run_ecm(df) for df in df_autoroutes]
results_routes_ecm = [run_ecm(df) for df in df_routes]

# Création des tableaux récapitulatifs des résultats pour chaque groupe de carburants
summary_table_autoroutes_ecm = summary_col(results_autoroutes_ecm, stars=True, model_names=labels,
                                           info_dict={'No. observations': lambda x: f"{int(x.nobs)}"})
summary_table_routes_ecm = summary_col(results_routes_ecm, stars=True, model_names=labels,
                                       info_dict={'No. observations': lambda x: f"{int(x.nobs)}"})

In [14]:
print("Résultats ECM pour les routes:")
display(summary_table_routes_ecm)
print("\nRésultats ECM pour les autoroutes:")
display(summary_table_autoroutes_ecm)

Résultats ECM pour les routes:


,Gazole,SP95,E10,SP98
delta_ln_Brent,0.1697***,0.2452***,0.1522***,0.1748***
,(0.0008),(0.0014),(0.0008),(0.0008)
delta_ln_Brent_1,0.2164***,0.1803***,0.1763***,0.1739***
,(0.0009),(0.0016),(0.0009),(0.0008)
delta_ln_Brent_2,0.1708***,0.1240***,0.1486***,0.1378***
,(0.0009),(0.0016),(0.0009),(0.0008)
delta_ln_PP_1,0.0073***,-0.0585***,-0.0342***,-0.0514***
,(0.0008),(0.0017),(0.0010),(0.0010)
error_correction,-0.0750***,-0.1417***,-0.0744***,-0.0803***
,(0.0003),(0.0009),(0.0004),(0.0004)



Résultats ECM pour les autoroutes:


,Gazole,SP95,E10,SP98
delta_ln_Brent,0.0650***,0.1475***,0.0884***,0.0843***
,(0.0027),(0.0219),(0.0026),(0.0024)
delta_ln_Brent_1,0.1657***,0.0821***,0.1579***,0.1443***
,(0.0028),(0.0239),(0.0026),(0.0025)
delta_ln_Brent_2,0.1697***,0.1190***,0.1702***,0.1580***
,(0.0028),(0.0234),(0.0027),(0.0025)
delta_ln_PP_1,0.0108***,0.0575*,-0.0941***,-0.0813***
,(0.0031),(0.0297),(0.0032),(0.0032)
error_correction,-0.0466***,-0.1015***,-0.0305***,-0.0301***
,(0.0009),(0.0124),(0.0008),(0.0008)


In [15]:
export_to_latex(summary_table_autoroutes_ecm, 'tex/ECM_A.tex')
export_to_latex(summary_table_routes_ecm, 'tex/ECM_R.tex')

Le fichier tex/ECM_A.tex a été créé avec succès.
Le fichier tex/ECM_R.tex a été créé avec succès.


### Relation de court-terme : Modèle à correction d'erreur avec asymétrie (Equation 3)

In [16]:
labels = ['Gazole', 'SP95', 'E10', 'SP98']

# Préparation des données pour la régression de correction d'erreur à court terme avec asymétrie
def prepare_data_for_ecm_asym(df):
    df = df.copy()
    
    # Calculer les différences et les termes de correction d'erreur
    df['delta_ln_PP'] = df.groupby('id')['ln_PP'].diff()
    df['delta_ln_Brent'] = df.groupby('id')['ln_Brent'].diff()
    df['error_correction'] = df.groupby('id')['residus'].shift(1)
    
    # Créer les variables indicatrices pour les ajustements asymétriques
    df['lambda'] = (df['error_correction'] > 0).astype(int)
    df['error_correction_pos'] = df['lambda'] * df['error_correction']
    df['error_correction_neg'] = (1 - df['lambda']) * df['error_correction']
    
    # Calculer les lags nécessaires
    df['delta_ln_PP_1'] = df.groupby('id')['delta_ln_PP'].shift(1)
    df['delta_ln_Brent_1'] = df.groupby('id')['delta_ln_Brent'].shift(1)
    df['delta_ln_Brent_2'] = df.groupby('id')['delta_ln_Brent_1'].shift(1)
    
    # Supprimer les lignes avec des valeurs manquantes après le diff() et shift()
    df.dropna(subset=['delta_ln_PP', 'delta_ln_Brent', 'error_correction',
                      'delta_ln_PP_1', 'delta_ln_Brent_1', 'delta_ln_Brent_2',
                      'error_correction_pos', 'error_correction_neg'], inplace=True)
    
    return df

# Fonction pour effectuer la régression de correction d'erreur à court terme avec asymétrie
def run_ecm_asym(df):
    df = prepare_data_for_ecm_asym(df)
    
    # Définir les variables pour le modèle, y compris les termes de correction d'erreur positifs et négatifs
    X_ecm = df[['delta_ln_Brent', 'delta_ln_Brent_1', 'delta_ln_Brent_2', 'delta_ln_PP_1', 
                'error_correction_pos', 'error_correction_neg']]
    y_ecm = df['delta_ln_PP']
    
    # Ajuster le modèle ECM sans constante
    model_ecm = sm.OLS(y_ecm, X_ecm)
    results_ecm = model_ecm.fit()
    return results_ecm

# Appliquer la fonction de régression ECM à chaque sous-groupe de DataFrame
results_autoroutes_ecm_asym = [run_ecm_asym(df) for df in df_autoroutes]
results_routes_ecm_asym = [run_ecm_asym(df) for df in df_routes]

# Créer les tableaux récapitulatifs des résultats avec asymétrie pour chaque groupe de carburants
summary_table_autoroutes_ecm_asym = summary_col(results_autoroutes_ecm_asym, stars=True, model_names=labels,
                                           info_dict={'No. observations': lambda x: f"{int(x.nobs)}"})
summary_table_routes_ecm_asym = summary_col(results_routes_ecm_asym, stars=True, model_names=labels,
                                       info_dict={'No. observations': lambda x: f"{int(x.nobs)}"})

In [17]:
print("Résultats ECM avec asymétrie pour les routes:")
display(summary_table_routes_ecm_asym)
print("\nRésultats ECM avec asymétrie pour les autoroutes:")
display(summary_table_autoroutes_ecm_asym)

Résultats ECM avec asymétrie pour les routes:


,Gazole,SP95,E10,SP98
delta_ln_Brent,0.1684***,0.2453***,0.1524***,0.1749***
,(0.0008),(0.0014),(0.0008),(0.0008)
delta_ln_Brent_1,0.2154***,0.1800***,0.1748***,0.1717***
,(0.0009),(0.0016),(0.0009),(0.0008)
delta_ln_Brent_2,0.1696***,0.1236***,0.1470***,0.1355***
,(0.0009),(0.0016),(0.0009),(0.0008)
delta_ln_PP_1,0.0051***,-0.0586***,-0.0342***,-0.0516***
,(0.0009),(0.0017),(0.0010),(0.0010)
error_correction_pos,-0.0659***,-0.1393***,-0.0630***,-0.0670***
,(0.0004),(0.0012),(0.0005),(0.0005)



Résultats ECM avec asymétrie pour les autoroutes:


,Gazole,SP95,E10,SP98
delta_ln_Brent,0.0634***,0.1468***,0.0884***,0.0844***
,(0.0027),(0.0219),(0.0026),(0.0024)
delta_ln_Brent_1,0.1625***,0.0764***,0.1578***,0.1443***
,(0.0028),(0.0240),(0.0026),(0.0025)
delta_ln_Brent_2,0.1672***,0.1158***,0.1703***,0.1579***
,(0.0028),(0.0234),(0.0027),(0.0025)
delta_ln_PP_1,0.0083***,0.0543*,-0.0939***,-0.0814***
,(0.0031),(0.0297),(0.0032),(0.0032)
error_correction_pos,-0.0331***,-0.0773***,-0.0322***,-0.0288***
,(0.0011),(0.0162),(0.0012),(0.0011)


In [18]:
export_to_latex(summary_table_autoroutes_ecm_asym, 'tex/ECM_asym_A.tex')
export_to_latex(summary_table_routes_ecm_asym, 'tex/ECM_asym_R.tex')

Le fichier tex/ECM_asym_A.tex a été créé avec succès.
Le fichier tex/ECM_asym_R.tex a été créé avec succès.


In [19]:
hypothesis='error_correction_pos = error_correction_neg'
test_results_r = []
for results in results_autoroutes_ecm_asym:
    test = results.t_test(hypothesis)
    test_results_r.append(test.pvalue.item())  # Utilisation de .item() pour obtenir un scalaire

# Test pour chaque modèle de résultats pour routes
test_results_a = []
for results in results_routes_ecm_asym:
    test = results.t_test(hypothesis)
    test_results_a.append(test.pvalue.item())  # Utilisation de .item() pour obtenir un scalaire

# Seuil de signification
alpha = 0.05

# Créer des DataFrames pour chaque groupe avec interprétation
wald_autoroutes = pd.DataFrame({
    'Carburant': labels,
    'P-valeur': test_results_r,
    'Interprétation': ['$\delta_3 \\ne \delta_4$' if p < alpha else '$\delta_3=\delta_4$' for p in test_results_r]
})

wald_routes = pd.DataFrame({
    'Carburant': labels,
    'P-valeur': test_results_a,
    'Interprétation': ['$\delta_3 \\ne \delta_4$' if p < alpha else '$\delta_3=\delta_4$' for p in test_results_a]
})

In [20]:
print("Résultats de test pour routes:")
display(wald_routes)
print("\nRésultats de test pour autoroutes:")
display(wald_autoroutes)

Résultats de test pour routes:


,Carburant,P-valeur,Interprétation
0,Gazole,9.215129e-226,$\delta_3 \ne \delta_4$
1,SP95,3.519170e-03,$\delta_3 \ne \delta_4$
2,E10,1.570063e-242,$\delta_3 \ne \delta_4$
3,SP98,0.000000e+00,$\delta_3 \ne \delta_4$



Résultats de test pour autoroutes:


,Carburant,P-valeur,Interprétation
0,Gazole,1.621660e-73,$\delta_3 \ne \delta_4$
1,SP95,2.022862e-02,$\delta_3 \ne \delta_4$
2,E10,4.960737e-02,$\delta_3 \ne \delta_4$
3,SP98,9.735865e-02,$\delta_3=\delta_4$


In [21]:
import pandas as pd

def calculate_pvalues(results_list, hypothesis):
    pvalues = []
    for results in results_list:
        test = results.t_test(hypothesis)
        pvalues.append(test.pvalue.item())  # Utilisation de .item() pour obtenir un scalaire
    return pvalues

# Hypothèse testée
hypothesis = 'error_correction_pos = error_correction_neg'

# Calcul des p-valeurs pour les deux ensembles de résultats
test_results_r = calculate_pvalues(results_routes_ecm_asym, hypothesis)
test_results_a = calculate_pvalues(results_autoroutes_ecm_asym, hypothesis)

# Seuil de signification
alpha = 0.05

# Création de l'index pour les données
index = pd.MultiIndex.from_product([labels, ['Route', 'Autoroute']], names=['Carburant', 'Type'])

# Préparation des données pour chaque type et chaque carburant
data = {
    'P-valeur': test_results_r + test_results_a,  # Concaténation des résultats routes puis autoroutes
    'Interprétation': ['$\delta_3 \\ne \delta_4$' if p < alpha else '$\delta_3=\delta_4$' for p in test_results_r] + 
                      ['$\delta_3 \\ne \delta_4$' if p < alpha else '$\delta_3=\delta_4$' for p in test_results_a]
}

# Création du DataFrame
wald_df = pd.DataFrame(data, index=index)

# Affichage du DataFrame
wald_df


P-valeur           Interprétation
Carburant Type                                             
Gazole    Route      9.215129e-226  $\delta_3 \ne \delta_4$
          Autoroute   3.519170e-03  $\delta_3 \ne \delta_4$
SP95      Route      1.570063e-242  $\delta_3 \ne \delta_4$
          Autoroute   0.000000e+00  $\delta_3 \ne \delta_4$
E10       Route       1.621660e-73  $\delta_3 \ne \delta_4$
          Autoroute   2.022862e-02  $\delta_3 \ne \delta_4$
SP98      Route       4.960737e-02  $\delta_3 \ne \delta_4$
          Autoroute   9.735865e-02      $\delta_3=\delta_4$

In [22]:
wald_df.to_latex('tex/wald_df')

In [23]:
wald_routes.to_latex('tex/wald_R.tex',index=False)
wald_autoroutes.to_latex('tex/wald_A.tex',index=False)